In [142]:
import requests
import mwparserfromhell
import csv
from bs4 import BeautifulSoup
import re
import validators

In [143]:
def fetch_pages(category_title):
    url = "https://bots.snpedia.com/api.php"
    params = {
        "action" : "query",
        "list" : "categorymembers",
        "cmtitle" : category_title,
        "cmlimit" : "max",
        "format" : "json",
    }

    response = requests.get(url, params=params)
    data = response.json()

    pages = data["query"]["categorymembers"]
    
    return pages

In [144]:
def fetch_and_process_page_content(page_title):
    content = fetch_page_content_by_title(page_title)
    # Process the content here (e.g., extract information, write to file, etc.)
    print(f"Content of page {page_title}: {content}\n")

In [145]:
def fetch_html(url):
    response = requests.get(url)
    html_content = response.content.decode("utf-8")
    return html_content

In [146]:
def write_to_json(content):
    with open('C:/Users/Gregorious/VSCode/Prometheus/SNPediaInfo.json', 'w') as f:
        json.dump(content, f)

In [147]:
def extract_rs_text(html):
    soup = BeautifulSoup(html, "html.parser")
    rs_text = None
    text_tags = soup.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6"])
    for tag in text_tags:
        text = tag.get_text().strip()
        if text.startswith("rs"):
            rs_text = text
            break
    return rs_text

In [148]:
def visit_rs_link(url):
    # Visit the URL and fetch the HTML content
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")

    # Process the content of the fetched webpage as needed
    # For example, you can print the page title
    page_title = soup.title.string
    
    #Looking for the body text of the page
    content_div = soup.find('div', {'id': 'mw-content-text'})

    if content_div:
        # Using a lambda function to find the parent or ancestor div with the 'mw-content-ltr' class
        parent_div = content_div.find_parent(lambda element: element and 'mw-content-ltr' in element.get('class', []))

        # Fallback to using the 'mw-content-text' div directly if the parent with the desired class is not found
        target_div = parent_div if parent_div else content_div

        # Extracting only the text within the <p> tags
        paragraphs = target_div.find_all('p')
        text = ' '.join(p.get_text(separator=' ', strip=True) for p in paragraphs)

        print(text)
    else:
        print("Content div not found.")


    #Looking for the table elements on the page
    table = soup.find('table', {'class': ['sortable', 'smwtable', 'jquery-tablesorter']})
    if table:
        # Extracting headers
        headers = []
        for th in table.find_all('th'):
            headers.append(th.text.strip())

        # Extracting rows
        rows = []
        for tr in table.find_all('tr')[1:]:  # Skipping the header row
            row = []
            for td in tr.find_all('td'):
                row.append(td.text.strip())
            rows.append(row)

        print("Headers:", headers)
        print("Rows:", rows)

pages = fetch_pages("Category:Is_a_snp")
page_titles = [page["title"] for page in pages]

for title in page_titles:
    # Construct the URL for the page title
    url = f"https://www.snpedia.com/index.php/{title.replace(' ', '_')}"

    # Fetch the rs link for the page
    rs_link = fetch_rs_link(url)

    # Visit the fetched link only if it's a valid URL
    if rs_link.startswith("http"):
        visit_rs_link(rs_link)

 One of the best-known genetic mutations, rs113993960 is also called delta508 or delta F508. It results in the loss of a phenylalanine (F) residue at amino acid position 508 of the cystic fibrosis CFTR gene. As first reported in 1989, about 70% of all cystic fibrosis patients carry this mutation. [ PMID 2567116 ] Cystic fibrosis is inherited in a recessive manner, so homozygotes for rs113993960 (-;-), and less commonly, compound heterozygotes carrying one rs113993960 (-) allele in addition to one other nonfunctional CFTR variant, are prone to developing cystic fibrosis . See also i3000001 and i5011261 (the designations used by 23andMe for this SNP).  [ PMID 15948195 ] Mutation spectrum in Jewish cystic fibrosis patients in Israel: implication to carrier screening.
Headers: ['Geno', 'Mag', 'Summary']
Rows: [['(-;-)', '8', 'Cystic Fibrosis; homozygote for delta F-508 mutation'], ['(-;CTT)', '3', 'carrier of a cystic fibrosis allele'], ['(CTT;CTT)', '0', 'normal']]
 [ PMID 18385274 ] This

In [ ]:
def fetch_rs_link(url):
    # Fetch the HTML content of the webpage
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")

    # Find the first link that contains "/index.php/Rs"
    rs_link = soup.find("a", href=re.compile("/index.php/Rs"))

    # If no link is found, return an error message
    if rs_link is None:
        return "No rs link found on the page"

    # Construct the full URL from the base URL and the relative link
    base_url = "https://www.snpedia.com"
    full_url = base_url + rs_link["href"]

    return full_url

pages = fetch_pages("Category:Is_a_snp")
page_titles = [page["title"] for page in pages]

for title in page_titles:
    # Construct the URL for the page title
    url = f"https://www.snpedia.com/index.php/{title.replace(' ', '_')}"

    # Fetch the rs link for the page
    rs_link = fetch_rs_link(url)

    # Print the result
    print(f"{title}: {rs_link}")